In [1]:
import numpy as np
import pandas as pd
import urllib3
from urllib.request import urlopen
import functions as fn
import xml.etree.ElementTree as ET
import ipywidgets as widgets
from ipywidgets import Layout,GridspecLayout,interact, interact_manual,VBox,HBox,Button, ButtonStyle, Output
from IPython.display import HTML, display, clear_output, FileLink, FileLinks

# [DataPipe Identification de notices de périos avec numéros isolés] 

***

## Méthodologie

L'objectif consiste à identifier dans le Sudoc pour chacun de nos rcr les lots de notices de périodiques ayant été exemplarisées avec des numéros isolés (c'est-à dire, selon nos pratiques locales, avec un champ d'exemplaire 999$p NI)

#### Etape 1 : extraction des sets de ppn par rcr

- Source de données : index Solr de l'application Périscope
- url racine de l'API Solr : https://periscope.sudoc.fr/SolrProxy
- construction des requêtes (arguments Solr normalisés)
 - q='930-b_t:VARIABLE_RCR AND 999-p_s:NI'
 - solrService=Pcp
 - fl=ppn_z,011-a_z,200-a_z,999-p_s,NbLocs_i *#sélection des champs souhaités dans la réponse Solr*
 - wt=json *#format de sortie pour le datapipe, affiche une erreur dans le navigateur (en mode affichage ne pas ajouter cet argument, le flux de données era en xml par défaut)*
 
Bien que l'API Solr renvoit l'intégralité des métadonnées de chaque notice (tous les champs bibs + tous les champs de tous les exemplares localisés), on ne peut se baser sur cette source car les valeurs des données d'exemplaires sont exposées de manière non stucturée (pour chaque sous-champs toutes les valeurs textuelles de tous les exemplaires sont concaténés en vrac dans des balises générqiue <str>).
 
#### Etape 2 : récupération des données d'exemplaires Sudoc pour chaque ppn

- Source de données : web service Abes Unimarc/MarcXML
- url de l'API : https://www.sudoc.fr/PPN.xml

***

In [3]:
output = widgets.Output()
spin = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    description='Harvesting:',
    bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': 'maroon'},
    orientation='horizontal'
)
filelinks_button = widgets.Button(description="Voir/Exporter le fichier résultat",
                                  button_style='success',
                                  layout = Layout(width='300px'))
filelinks_output = widgets.Output()
def on_flbutton_clicked(b):
    with filelinks_output:
        display(HTML("<p>Visualiser les résultats : cliquer sur le lien</p><p>Exporter le fichier : faire un clic droit puis Enregistrer la cible du lien sous</p>"))
        display(FileLinks("result_files/"))
filelinks_button.on_click(on_flbutton_clicked) 

In [12]:
def harvest_solr_periscope(rcr,total):
    """Request Periscope Solr API on a specific rcr and get all records by looping on 10 records sets
    ------------------
     Parameters:
           rcr : str, the rcr of the selected library (cf button)
           total : int, the value of the numFound key in the first periscope API request's response (cf button)
    ------------------
       Result: a dataframe with the following columns (ppn_z,011-a_z,200-a_z,999-p_s,NbLocs_i)
    """
    with output:
        clear_output()
        df = pd.DataFrame()
        for i in range(0,total,10):
            result = fn.periscope_api(rcr,i,10)['response']['docs'] 
            df_temp = pd.DataFrame(result)
            df = df.append(df_temp)
        spin.value=3
        return create_ppns_file(df,rcr)

In [13]:
def create_ppns_file(df,rcr):
    """Get only the records ppn and store them in a static xml file 
    ------------------
     Parameters:
           df : dataframe, the current dataframe
           rcr : str, the rcr of the selected library (cf button)
    ------------------
       Result: call to the exec_w function
    """
    list_ppn = df['ppn_z'].tolist()
    root = ET.Element("root")
    for ppn in list_ppn:
        ET.SubElement(root, "ppn").text = str(ppn)
    tree = ET.ElementTree(root)
    tree.write(fn.file_path("temporary_files/ppns.xml"))
    spin.value=5
    display(filelinks_button,filelinks_output,fn.exec_w(df,rcr))
    spin.value=10

In [8]:
def display_html_header(data):
    """First process  : display the libraries widgets and configure the parameters to pass on the on_click function
    ------------------
     Parameters:
           data : dict, the manual-made list of libraries and rcr
    ------------------
       Result: html blocks
    """
    button = widgets.Button(description="Obtenir les notices",button_style='primary')
    def on_button_clicked(b):
        harvest_solr_periscope(str(data["rcr"]),int(data["count"]))        
    button.on_click(on_button_clicked)
    return display(HTML('<td><h4>'+data["name"]+' (rcr '+data["rcr"]+')</h4><p>Nombre de notices : '+str(data["count"])+'</p>'),button,HTML("</td>")) 

*Le nombre de notices mentionné par rcr est un maximum donné à titre indicatif, il résulte du comptage pour chaque ppn ayant une localisation pour le rcr concerné de tous les exemplaires ayant une zone 999\\$p NI mais sans pouvoir être sûr, du fait de la structure des données de l'API Solr, que ce champ 999 soit bien complété par un sous-champ 999\\$5 RCR.*

*Le filtrage sur les exemplaires du rcr est effectué lors de l'étape 2 du workflow.*

In [10]:
display(HTML("<table><tr>"))
data_bib =[
 {'rcr': '060882103', 'name': 'BU Droit', 'count': fn.periscope_api('060882103',0,1)['response']['numFound']},
 {'rcr': '060882104', 'name': 'BU Lettres', 'count': fn.periscope_api('060882104',0,1)['response']['numFound']},
 {'rcr': '060882101', 'name': 'BU Sciences', 'count': fn.periscope_api('060882101',0,1)['response']['numFound']},
 {'rcr': '060882102', 'name': 'BU Médecine', 'count': fn.periscope_api('060882102',0,1)['response']['numFound']},
 {'rcr': '060882105', 'name': 'BU SJA', 'count': fn.periscope_api('060882105',0,1)['response']['numFound']}]
for item in data_bib:
    display_html_header(item)
display(HTML("</table>"))

Button(button_style='primary', description='Obtenir les notices', style=ButtonStyle())

Button(button_style='primary', description='Obtenir les notices', style=ButtonStyle())

Button(button_style='primary', description='Obtenir les notices', style=ButtonStyle())

Button(button_style='primary', description='Obtenir les notices', style=ButtonStyle())

Button(button_style='primary', description='Obtenir les notices', style=ButtonStyle())

***

In [9]:
display(spin,output)

IntProgress(value=0, bar_style='success', description='Harvesting:', max=10, style=ProgressStyle(bar_color='ma…

Output()